In [1]:
import pandas as pd
import sqlite3
import streamlit as st

In [2]:
%store -r data



print(data)

{'city': ['none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none'], 'days': ['30+', '30+', '10-14.5', '15-19.5', '20-36.5', '37+', 'till 19.5', '20-36.5', '37+', '20+', '20+', '20+', '60+'], 'rights': ['טיפול 1500 ש"ח החזר', 'טיפול זוגי 1500 ש"ח החזר', '726 מענק אוטומטי', '2904 מענק אוטומטי', '6534 מענק אוטומטי', '11616 מענק אוטומטי', 'M1 הנחה של 59 שקלים לאגרת הרישוי השנתית של רכב עבור אופנוע/מרכב מסוג', 'M1 הנחה של 119 שקלים לאגרת הרישוי השנתית של רכב עבור אופנוע/מרכב מסוג', 'M1 הנחה של 119 שקלים לאגרת הרישוי השנתית של רכב עבור אופנוע/מרכב מסוג', '12 שקלים עבור חידוש רישיון נהיגה', 'הנחה של 5% בתשלום הארנונה', 'זכאות לדיור במחיר מופחת-מחיר למשתכן על ידי עמידר,מגער ומגלם', 'שובר של 1500 שקלים שנשלח אוטומטית']}


In [3]:
import sqlite3

def insert_and_print_data():
    # Connect to SQLite database with a timeout to avoid locking issues
    conn = sqlite3.connect('questionnaire.db', timeout=10)
    c = conn.cursor()

    try:
        # Drop and recreate tables to ensure fresh start for demonstration
        c.execute('DROP TABLE IF EXISTS users;')
        c.execute('DROP TABLE IF EXISTS children;')
        
        c.execute('''
            CREATE TABLE users (
                id INTEGER PRIMARY KEY,
                first_name TEXT,
                last_name TEXT,
                date_of_birth TEXT,
                gender TEXT,
                marital_status TEXT,
                nationality TEXT,
                street_address TEXT,
                city TEXT,
                state_province TEXT,
                postal_code TEXT,
                country TEXT,
                phone_number TEXT,
                email_address TEXT,
                num_children INTEGER,
                armyid TEXT,
                other_first_name TEXT,
                other_last_name TEXT,
                total_days INTEGER
            )
        ''')

        c.execute('''
            CREATE TABLE children (
                id INTEGER PRIMARY KEY,
                parent_id INTEGER,
                child_name TEXT,
                child_age INTEGER,
                child_gender TEXT,
                child_adopted BOOLEAN,
                child_lives_home BOOLEAN,
                FOREIGN KEY(parent_id) REFERENCES users(id)
            )
        ''')

        # Inserting example user data
        c.execute('''
            INSERT INTO users (first_name, last_name, date_of_birth, gender, marital_status, nationality,
                street_address, city, state_province, postal_code, country,
                phone_number, email_address, num_children, armyid, other_first_name, other_last_name, total_days)
            VALUES ('Jane', 'Doe', '1990-01-01', 'Female', 'Married', 'Israeli', 
                '123 Main St', 'Tel Aviv', 'TA', '12345', 'Israel', 
                '+1234567890', 'jane.doe@example.com', 2, 'ABC123', 'John', 'Doe', 500)
        ''')

        # Capture the parent_id of the last inserted user
        parent_id = c.lastrowid

        # Inserting example children data
        c.executemany('''
            INSERT INTO children (parent_id, child_name, child_age, child_gender, child_adopted, child_lives_home)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', [
            (parent_id, 'Child1', 8, 'Male', False, True),
            (parent_id, 'Child2', 5, 'Female', False, True)
        ])

        conn.commit()

        # Print users data
        c.execute('SELECT * FROM users')
        users = c.fetchall()
        print("Users Data:")
        for user in users:
            print(user)

        # Print children data
        c.execute('SELECT * FROM children')
        children = c.fetchall()
        print("Children Data:")
        for child in children:
            print(child)

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

# Call the function to insert and print data
insert_and_print_data()


Users Data:
(1, 'Jane', 'Doe', '1990-01-01', 'Female', 'Married', 'Israeli', '123 Main St', 'Tel Aviv', 'TA', '12345', 'Israel', '+1234567890', 'jane.doe@example.com', 2, 'ABC123', 'John', 'Doe', 500)
Children Data:
(1, 1, 'Child1', 8, 'Male', 0, 1)
(2, 1, 'Child2', 5, 'Female', 0, 1)
